# Analysis of the related datasets texts

In this notebook, we will compare the texts of the related datasets.

Essentially, we will compare the text labels from the related datasets and what was assigned in OLID-BR.

In [1]:
import sys
from pathlib import Path

if str(Path(".").absolute().parent) not in sys.path:
    sys.path.append(str(Path(".").absolute().parent.parent))

In [2]:
from dotenv import load_dotenv

# Initialize the env vars
load_dotenv("../../.env")

True

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from src.settings import AppSettings
from src.kaggle import download_dataset

sns.set_theme(
    style="white",
    rc={
        "axes.spines.right": False,
        "axes.spines.top": False
    }
)

args = AppSettings()

## Download OLID-BR dataset

In [4]:
files = download_dataset(
    output_files=[
        "train.csv",
        "test.csv",
        "train.json",
        "test.json"
    ]
)

In [5]:
df_train = files["train.csv"]

print(f"Shape: {df_train.shape}")
df_train.head()

Shape: (4765, 17)


,id,text,is_offensive,is_targeted,targeted_type,toxic_spans,health,ideology,insult,lgbtqphobia,other_lifestyle,physical_aspects,profanity_obscene,racism,religious_intolerance,sexism,xenophobia
0,430b13705cf34e13b74bc999425187c3,USER USER é muito bom. USER ^^ E claro a equip...,NOT,UNT,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False
1,c779826dc43f460cb18e8429ca443477,Pior do que adolescentezinhas de merda...são p...,OFF,UNT,NaN,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",False,False,True,False,False,False,True,False,False,True,False
2,e64148caa4474fc79298e01d0dda8f5e,USER Toma no cu é vitamina como tu e tua prima.,OFF,TIN,GRP,"[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...",False,False,True,False,False,False,True,False,False,False,False
3,cc66b54eeec24607a67e2259134a1cdd,"Muito bom, pena a circunstâncias serem ruins, ...",OFF,UNT,NaN,"[119, 120, 121, 122, 123, 124, 125, 126, 127, ...",False,False,True,False,False,False,False,False,False,False,False
4,a3d7839456ae4258a70298fcf637952e,"Podia ter beijo também, pra ver se o homofóbic...",OFF,UNT,NaN,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...",False,False,True,False,False,False,False,False,False,False,False


In [6]:
full_data = files["train.json"] + files["test.json"]

print(f"Count: {len(full_data)}")

Count: 6354


In [7]:
from typing import List, Dict

def get_origin_text(text: str, data: List[Dict]):
    for item in data:
        if item["text"] == text:
            return item["metadata"]["source"]

rows = []
for item in df_train.to_dict(orient="records"):
    rows.append(
        {
            "id": item["id"],
            "text": item["text"],
            "is_offensive": item["is_offensive"],
            "source": get_origin_text(item["text"], full_data)
        }
    )

df_source = pd.DataFrame(rows)

print(f"shape: {df_source.shape}")
df_source.head()

shape: (4765, 4)


,id,text,is_offensive,source
0,430b13705cf34e13b74bc999425187c3,USER USER é muito bom. USER ^^ E claro a equip...,NOT,YouTube
1,c779826dc43f460cb18e8429ca443477,Pior do que adolescentezinhas de merda...são p...,OFF,YouTube
2,e64148caa4474fc79298e01d0dda8f5e,USER Toma no cu é vitamina como tu e tua prima.,OFF,Twitter
3,cc66b54eeec24607a67e2259134a1cdd,"Muito bom, pena a circunstâncias serem ruins, ...",OFF,YouTube
4,a3d7839456ae4258a70298fcf637952e,"Podia ter beijo também, pra ver se o homofóbic...",OFF,YouTube


In [8]:
df_temp = df_source[
    ["text",
    "source",
    "is_offensive"
    ]
].groupby(["source", "is_offensive"]).agg(
    count=("text", "count")
)

df_temp.transpose()

source       HLPHSD     NCCVG     OffComBR     ToLD-BR      Twitter        \
is_offensive    NOT OFF   NOT OFF      NOT OFF     NOT  OFF     NOT   OFF   
count             8  46     1  36        1   8      19  310     111  1718   

source       YouTube        
is_offensive     NOT   OFF  
count            333  2174

In [9]:
df_olidbr_related_datasets = df_source[df_source["source"].isin(["HLPHSD", "NCCVG", "OffComBR", "ToLD-BR"])]
df_olidbr_related_datasets[["text", "source"]].groupby("source").count()

,text
source,
HLPHSD,54
NCCVG,37
OffComBR,9
ToLD-BR,329


In [10]:
df_olidbr_related_datasets["is_offensive"].value_counts()

OFF    400
NOT     29
Name: is_offensive, dtype: int64

## Download related datasets

In [11]:
import io
import requests
from typing import Any, Dict, List

def get_file_github(
    url: str,
    extension: str = "csv",
    pd_args: Dict[Any, Any] = {},
    return_as_dict: bool = True) -> List[Any] | io.StringIO:
    """
    Download a file from a github url

    Args:
    - url: the url to the file.
    - extension: the extension of the file.
    - pd_args: the arguments to pass to the pandas read function.
    - return_as_dict: if the function should return a dict or a file object.

    Returns:
    - pd.DataFrame: the dataframe of the file
    """
    download = requests.get(url).content
    
    download = io.StringIO(download.decode("utf-8"))

    if return_as_dict:
        if extension.lower() in ["csv", "arff"]:
            return pd.read_csv(download, **pd_args).to_dict(orient="records")
        elif "json" in extension.lower():
            return pd.read_json(download, **pd_args).to_dict(orient="records")
        else:
            raise ValueError("Only csv, arff and json are supported")
    else:
        return download

### rogersdepelle/OffComBR

In [12]:
import re

def format_offcombr(raw: List[str]):
    rows = []
    for line in raw:
        if line.startswith("yes") or line.startswith("no"):
            line = line.split(",")
            rows.append(
                {
                    "text": re.sub(r"\'(.*)\'\r\n", r"\1", line[1]).strip(),
                    "is_offensive": "OFF" if line[0] == "yes" else "NOT"
                }
            )
    return rows

In [13]:
url = "https://raw.githubusercontent.com/rogersdepelle/OffComBR/master/OffComBR2.arff"

offcombr = get_file_github(
    url,
    extension="arff",
    return_as_dict=False
)

offcombr = format_offcombr(offcombr.readlines())

print(f"Count: {len(offcombr)}")

offcombr[:5]

Count: 1250


[{'text': 'Votaram no PEZAO Agora tomem no CZAO', 'is_offensive': 'OFF'},
 {'text': 'cuidado com a poupanca pessoal Lembram o que aconteceu na epoca do Collor ne',
  'is_offensive': 'NOT'},
 {'text': 'Sabe o que eu acho engracado os nossos governantes  nao pensam em cortar regalias e beneficios desnecessarios que os favorecem porque sera ne e mais do que claro ate mesmo para quem nao quer enxergar eles sao estao la para defender seus proprios interesses  e os dos empresario o no casso eles tambem ou comecamos a tomar uma atitude para mudar de uma vez por todas essa roubalheira nesse pais o a tendencia e so piorar para o povo porque dinheiro para investimentos nao tem mais para aumentos de salario e regalias nao falta',
  'is_offensive': 'NOT'},
 {'text': 'os cariocas tem o que merecem um pessoal que so sabem toma banho de sol e pratica a violencia e nao deu outra de onde se tira e nao coloca um dia acaba',
  'is_offensive': 'OFF'},
 {'text': 'Podiam retirar dos lucros dos bancos', 'is_

### LaCAfe/Dataset-Hatespeech

In [14]:
def format_nccvg(raw: List[dict]):
    rows = []
    for item in raw:
        rows.append(
            {
                "text": item["txt"],
                "is_offensive": "OFF" if item["has_anger"] == "S" else "NOT"
            }
        )
    return rows

In [15]:
url = "https://raw.githubusercontent.com/LaCAfe/Dataset-Hatespeech/master/data/df_dataset.json"

nccvg = get_file_github(url, extension="json")
nccvg = format_nccvg(nccvg)

print(f"Count: {len(nccvg)}")
nccvg[:5]

Count: 7672


[{'text': '>>22994apóio o >>22995. um passo de cada vez. não tenha pressa, mas sempre movendo pra frente. e não tenha medo de pedir ajuda a amigos, parentes ou psicólogos. não tem nada de mais.e outra: faculdade nem sempre é a solução. tem cursos técnicos que também contam como superior e que às vezes são mais curtos que faculdades e te pagam melhor.',
  'is_offensive': 'OFF'},
 {'text': 'eu ainda vou surtar com essa fic', 'is_offensive': 'NOT'},
 {'text': '>>11826 puta merda… se alguém de fato analisar esse chart…. saberá o motivo que o mundo ta na merda.',
  'is_offensive': 'OFF'},
 {'text': 'seguinte, fizemos 4 anos de namoro semana passada, mas nós estávamos brigados e não comemoramos nada. fiquei sabendo que, no dia do nosso aniversário de namoro, ela tinha saído com um amigo. não dei a miníma foda para isso, afinal de contas eu conhecia o cara e etc. fizemos as pazes e ela voltou muito desconfiada, até que ontem ela veio dormir aqui em casa, o que acontece é que hoje pela manhã e

### paulafortuna/Portuguese-Hate-Speech-Dataset

In [16]:
def format_hlphsd(raw: List[dict]):
    rows = []
    for item in raw:
        rows.append(
            {
                "text": item["text"],
                "is_offensive": "OFF" if item["hatespeech_comb"] == 1 else "NOT"
            }
        )
    return rows

In [17]:
url = "https://raw.githubusercontent.com/paulafortuna/Portuguese-Hate-Speech-Dataset/master/2019-05-28_portuguese_hate_speech_binary_classification.csv"

hlphsd = get_file_github(url, extension="csv")
hlphsd = format_hlphsd(hlphsd)

print(f"Count: {len(hlphsd)}")
hlphsd[:5]

Count: 5670


[{'text': '@__andrea__b \nO cara vive em outro mundo\nNão no mundo real\nREFUGIADOS são os que vivem\nNas favelas vizinhas as suas fortalezas',
  'is_offensive': 'OFF'},
 {'text': '@_carmeloneto Estes incompetentes não cuidam nem do povo brasileiro e nem dos poucos refugiados que aqui estão.',
  'is_offensive': 'NOT'},
 {'text': "@_carmeloneto \nOs 'cumpanhero' quebraram todas as regras.",
  'is_offensive': 'NOT'},
 {'text': '@_GlitteryKisses é isso não conseguem pensar no sentido lato para além do que se vê a frente dos olhos',
  'is_offensive': 'NOT'},
 {'text': '@_iglira bom dia macaco branco haha', 'is_offensive': 'OFF'}]

### JAugusto97/ToLD-Br

In [18]:
def format_toldbr(raw: List[dict]):
    rows = []
    for item in raw:
        row = {
            "text": item["text"],
            "is_offensive": "NOT"
        }

        for value in item.values():
            if isinstance(value, float) and value > 0.0:
                row["is_offensive"] = "OFF"
                break
        rows.append(row)

    return rows

In [19]:
url = "https://raw.githubusercontent.com/JAugusto97/ToLD-Br/main/ToLD-BR.csv"

told_br = get_file_github(url, extension="csv")
told_br = format_toldbr(told_br)

print(f"Count: {len(told_br)}")
told_br[:5]

Count: 21000


[{'text': 'Meu nivel de amizade com isis é ela ter meu insta e eu ter o dela, e quando eu penso que não ela manda mensagem “ falano otario ta falando dnv no insta”',
  'is_offensive': 'OFF'},
 {'text': 'rt @user @user o cara adultera dados, que foram desmascarados e ainda quer ficar no governo?',
  'is_offensive': 'OFF'},
 {'text': '@user @user @user o cara só é simplesmente o maior vencedor da história de futebol, tá com 36 anos e tem gás demais e não um gordo com joelho fodido',
  'is_offensive': 'OFF'},
 {'text': 'eu to chorando vei vsf e eu nem staneio izone nem nada https://t.co/rglb8luutw',
  'is_offensive': 'OFF'},
 {'text': 'Eleitor do Bolsonaro é tão ignorante q não percebeu q a frase abaixo significa o seguinte:\n\n“É melhor falar um monte de bosta do que ficar calado”.\n\nAinda transformaram em imagem bonitinha com citação e data hahhahahahhahahhahah. https://t.co/cBpSJT1Gdf',
  'is_offensive': 'OFF'}]

## Analysis

In [20]:
from src.anonymize import Anonymizer

anonymizer = Anonymizer()

In [21]:
related_datasets = []

for source, dataset in {
    "OffComBR": offcombr,
    "NCCVG": nccvg,
    "HLPHSD": hlphsd,
    "ToLD-BR": told_br
}.items():
    for item in dataset:
        item["source"] = source
        item["text"] = anonymizer.remove_users(item["text"])
        item["text"] = anonymizer.remove_urls(item["text"])
        item["text"] = anonymizer.remove_hashtags(item["text"])
        related_datasets.append(item)

print(f"Count: {len(related_datasets)}")

Count: 35592


In [22]:
def get_label_from_olidbr(text: str, data: pd.DataFrame):
    try:
        return data[data["text"] == text]["is_offensive"].values[0]
    except:
        return None

df_related_datasets = pd.DataFrame(related_datasets)

df_related_datasets["is_offensive_olidbr"] = df_related_datasets["text"].apply(
    lambda x: get_label_from_olidbr(x, df_olidbr_related_datasets)
)

df_related_datasets.rename(
    columns={"is_offensive": "is_offensive_original"},
    inplace=True
)

df_related_datasets["is_offensive_olidbr"].value_counts()

OFF    331
NOT     21
Name: is_offensive_olidbr, dtype: int64

In [23]:
import warnings

df_temp = df_related_datasets[~df_related_datasets["is_offensive_olidbr"].isna()]

# get match_label
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    df_temp["match_label"] = df_temp.apply(
        lambda x: 1 if x["is_offensive_original"] == x["is_offensive_olidbr"] else 0,
        axis=1
    )

df_temp["match_label"].value_counts()

1    253
0     99
Name: match_label, dtype: int64

In [24]:
print(f"Accuracy: {df_temp['match_label'].value_counts()[1] / len(df_temp):.2f}")

Accuracy: 0.72
